# Dolorem Ipsum

## Prep script

# https://www.biblestudyguide.org/ebooks/bibles/vul.pdf


something


In [77]:
import random;
import json;

n = 2;
usewds = []
start = []
recent = []
rem=''
mark = {}
wordlist = [set(),set()]

with open('./vulgate.txt', mode="rb") as f:
    while True:
        k = f.read(10)
        if len(k) == 0: 
            if len(start) > 0:
                usewds = start[:]
                start.clear()
            else:
                break;
        s = rem + str(k, encoding='utf-8')
        i=s.rindex(' ')
        div = s[:i]
        rem = s[i:]
        wds = usewds or div.split(" ")
        for w in wds:
            if w=='': continue;
            if len(start) < n and len(k) > 0: start.append(w)
            if len(recent) == n:
                a = recent[-2]
                b = recent[-1]
                if not (a in mark): 
                    mark[a] = {}
                if not (b in mark[a]): 
                    fresh = [set(), set()]
                    mark[a][b] = fresh
                try:
                    z = 0 if w[1] in ['a', 'e', 'i', 'o', 'u'] else 1
                    mark[a][b][z].add(w)
                    wordlist[z].add(w)
                except: print(w)
            recent.append(w)
            if len(recent) > n: recent.pop(0)
wordlist[0] = list(wordlist[0])
wordlist[1] = list(wordlist[1])

mark = {a: {c: [list(e) for e in d] for c, d in b.items()} for a, b in mark.items()}

out = open('./words.json', 'w')
json.dump({'mark':mark,'wordlist':wordlist}, out)
out.close()

print("ready")

ready


In [147]:
#in practice this step would be manual
import shutil
shutil.copyfile('message.txt', 'markov.txt')

'markov.txt'

In [139]:
# Download source markov chain db
import urllib.request
u = 'https://ibiwan.com/static/words.json'
f = open ('./markov.txt', 'a');
for line in urllib.request.urlopen(u):
    f.write(line.decode('utf-8'))
f.close()

# 
m, v = open('markov.txt', 'r', encoding='utf-8').read().split("{",1);
k = json.loads('{'+v)
mark = k['mark']
wordlist = k['wordlist']

import os
# recover word database space
os.remove('markov.txt')


w1 = random.choice(list(mark.keys()))
w2 = random.choice(list(mark[w1].keys()))
bites = [bin(ord(b))[2:].rjust(8, '0') for b in m]
bs = ''.join(bites)
gen = [w1, w2]
out = []
sentenceLength = 0
minSentenceLength = 5
minClauseLength = 6
for c in bs:
    branch = int(c)
    [a, b] = gen[-2:]
    cands = []
    outWord = ''
    vecWord = ''
    if a in mark and b in mark[a]:
        cands = list(mark[a][b][branch])
        if len(cands):
            vecWord = random.choice(cands)
            outWord = vecWord
        else:
            vecWord = random.choice(list(mark[a][b][1 - branch]))
            outWord = random.choice(wordlist[branch])
    elif a in mark:
        print(a, b, outWord, vecWord, mark[a][branch].keys(),mark[a][1 - branch].keys(), 'GEN', gen)
        print(mark[a])
        break
    else:
        outWord = random.choice(cands) if len(cands) > 0 else random.choice(wordlist[1-branch])
        vecWord = random.choice(list(mark[a][b][1-branch]))

    endSen = False
    endClause = False
    if sentenceLength > minSentenceLength:
        for j in range(sentenceLength - minSentenceLength):
            br = random.randint(0,5) == 0
            if br:
                endSen = True
                break;
    if sentenceLength == 0:
        outWord = outWord.capitalize()
    if sentenceLength > minClauseLength:
        for k in range(sentenceLength - minClauseLength):
            br = random.randint(0, 1) == 0
            if br:
                endClause = True
                break;
    if endSen:
        out.append(outWord + '.')
        sentenceLength = 0
    elif endClause:
        out.append(outWord + ',')
        sentenceLength = 1
    else:
        out.append(outWord)
        sentenceLength += 1
    gen = [b, vecWord]
lorem = " ".join(out) + '.'

out = open('./lorem.txt', 'w')
out.write(lorem)
out.close()

In [140]:
lorem = open('./lorem.txt','r').read()
bits = ["0" if word[1] in ['a', 'e', 'i', 'o', 'u'] else "1" for word in lorem.split(" ")]
message = ""
while len(bits) > 0:
    bitbyte = bits[0:8]
    bites = ''.join(bitbyte)
    val = int(bites, 2)
    message += chr(val)
    bits = bits[8:]
print(message)


Do it tomorrow night!
Signed, Me.
